In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
df = pd.read_csv("/content/train.csv").drop(columns=['id'])
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,label
0,4,99,76,15,51,23.2,0.223,21,0
1,0,102,52,0,0,25.1,0.078,21,0
2,3,126,88,41,235,39.3,0.704,27,0
3,5,109,75,26,0,36.0,0.546,60,0
4,4,156,75,0,0,48.3,0.238,32,1
...,...,...,...,...,...,...,...,...,...
595,0,91,68,32,210,39.9,0.381,25,0
596,5,95,72,33,0,37.7,0.370,27,0
597,1,124,60,32,0,35.8,0.514,21,0
598,9,156,86,28,155,34.3,1.189,42,1


In [ ]:
X = df.drop('label', axis=1).values
y = df['label'].values

In [ ]:
RANDOM_SEED = 42

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state = RANDOM_SEED)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
class ANN_Model(nn.Module):
  def __init__(self, input_features=8, hidden1 = 128, hidden2 = 128, hidden3 = 128, hidden4 =128, output_features = 2):
    super(ANN_Model, self).__init__()

    self.layer1 = nn.Linear(input_features, hidden1)
    self.layer2 = nn.Linear(hidden1, hidden2)
    self.layer3 = nn.Linear(hidden2, hidden3)
    self.layer4 = nn.Linear(hidden3, hidden4)
    self.layer5 = nn.Linear(hidden4, output_features)

    self.bn1 = nn.BatchNorm1d(hidden1)
    self.bn2 = nn.BatchNorm1d(hidden2)
    self.bn3 = nn.BatchNorm1d(hidden3)
    self.bn4 = nn.BatchNorm1d(hidden4)

  def forward(self, x):
      x = F.relu(self.layer1(x))
      x = F.relu(self.layer2(x))
      x = F.relu(self.layer3(x))
      x = F.relu(self.layer4(x))
      x = self.layer5(x)
      return x


In [ ]:
torch.manual_seed(3)
model = ANN_Model()

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 200
final_losses = []
for i in range(epochs):
  i=i+1
  y_pred = model.forward(X_train)
  loss = loss_function(y_pred, y_train)
  final_losses.append(loss.item())
  print("Epoch: {} and the loss: {}".format(i, loss.item()))
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

Epoch: 1 and the loss: 0.839372456073761
Epoch: 2 and the loss: 1.0381025075912476
Epoch: 3 and the loss: 0.8473703265190125
Epoch: 4 and the loss: 0.6538954377174377
Epoch: 5 and the loss: 0.7119033932685852
Epoch: 6 and the loss: 0.7359218001365662
Epoch: 7 and the loss: 0.6796057224273682
Epoch: 8 and the loss: 0.636618435382843
Epoch: 9 and the loss: 0.6392059922218323
Epoch: 10 and the loss: 0.6533602476119995
Epoch: 11 and the loss: 0.6477664113044739
Epoch: 12 and the loss: 0.6320392489433289
Epoch: 13 and the loss: 0.6240158677101135
Epoch: 14 and the loss: 0.6267880797386169
Epoch: 15 and the loss: 0.6264992356300354
Epoch: 16 and the loss: 0.6175623536109924
Epoch: 17 and the loss: 0.6094179749488831
Epoch: 18 and the loss: 0.6108213067054749
Epoch: 19 and the loss: 0.6125670075416565
Epoch: 20 and the loss: 0.6049405932426453
Epoch: 21 and the loss: 0.5978125333786011
Epoch: 22 and the loss: 0.5983760356903076
Epoch: 23 and the loss: 0.5984914898872375
Epoch: 24 and the loss

In [ ]:
# Prediction in X test data
predictions = []
with torch.no_grad():
  for i, data in enumerate(X_test):
    y_pred = model(data)
    predictions.append(y_pred.argmax().item())

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,predictions)
score

0.7

In [ ]:
test_df = pd.read_csv("/content/test.csv").drop(columns=['id'])

In [ ]:
new_data = torch.FloatTensor(test_df.values)

In [ ]:
prediction=[]
with torch.no_grad():
    for i,data in enumerate(new_data):
        pred=model(data)
        prediction.append(pred.argmax().item())

In [ ]:
ids = list(range(1, len(prediction) + 1))
result_df = pd.DataFrame({'id': ids, 'label': prediction})
result_df.to_csv("/content/predictions2.csv", index=False)